In [3]:
%run main.py

In [4]:
product_iri = ProductIRI("https://vocab.sentier.dev/products/")

In [5]:
amount = 7

In [6]:
user_input = UserInput(
    product_iri=product_iri,
    unit = 'kg',
    amount=amount
)

In [ ]:
user_input

In [ ]:
run_config = RunConfig()

In [ ]:
crop = Crop(user_input = user_input, run_config = run_config)

In [ ]:
resulting_demands = crop.run()

In [ ]:
resulting_demands

# Main tst SentierModel

In [ ]:
import sentier_data_tools

In [ ]:
sentier_data_tools.__version__

In [ ]:
!pip install sentier_data_tools==0.3.0

In [ ]:
%run main_tst.py

# Data.py

In [ ]:
import data

In [ ]:
data.create_example_local_datastorage()